In [ ]:
# 스트림

import gradio as gr
import random
import time

import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

def create_generator(text):
    gen = openai.chat.completions.create(
      model="gpt-4o-mini",
      messages=[
        {
          "role": "system",
          "content": "당신은 친절한 어시스턴트입니다. 사용자의 질문에 친절하게 대답하세요."
        },
        {
          "role": "user",
          "content": text
        }
      ],
      temperature=0.5,
      max_tokens=512,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0,
      stream=True
    )
    return gen

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")

    def user(user_message, history):
        return "", history + [[user_message, None]]

    def bot(history):
        gen = create_generator(history[-1][0])
        history[-1][1] = ""
        while True:
            response = next(gen)
            delta = response.choices[0].delta
            if delta.content is not None:
                history[-1][1] += delta.content
            else:
                break
            yield history

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    clear.click(lambda: None, None, chatbot, queue=False)
    
demo.queue()
demo.launch(server_name='0.0.0.0')

In [ ]:
import openai
import os
import gradio as gr

openai.api_key = os.getenv("OPENAI_API_KEY")

def chatgpt_respond(message, chat_history):
    messages = [{"role": "system", "content": "넌 불친절한 챗봇이야"}]
    for user_msg, bot_msg in chat_history:
        messages.append(
            {"role": "user", "content": user_msg}
            )
        if bot_msg is not None:
            messages.append(
                {"role": "assistant", "content": bot_msg}
                )

    messages.append({"role": "user", "content": message})

    response = openai.chat.completions.create(
        model="gpt-4o", 
        messages=messages, 
        stream=True
        )
    bot_message = ""

    for gen in response:
        delta = getattr(gen.choices[0].delta, 'content', None)
        if delta:
            bot_message += delta
            yield message, chat_history + [(message, bot_message)]
    
    chat_history.append((message, bot_message))
    yield "", chat_history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])
    msg.submit(chatgpt_respond, [msg, chatbot], [msg, chatbot])

demo.launch(server_name="0.0.0.0")
